# Proyecto del Día 15 - Optimizar Modelos de Machine Learning

El proyecto del día de hoy consiste en tomar un código que ya está desarrollado con Scikit-Learn, y modificarlo agregando varios apspectos que hemos aprendido el día de hoy.

Este programa es un simple análisis usando el modelo de **Bosques Aleatorios** en un dataset de **Scikit-Learn** sobre registros médicos de **pacientes con diabetes**.

### Consigna

Tu trabajo aquí simplemente consiste en ejecutar este código, y analizarlo para identificar qué hace cada línea, y cuál es su función dentro del código. Una vez que te hayas asegurado de haber identificado sus elementos y de haber comprendido su funcionamiento en general, realiza todas las modificaciones que creas necesarias, para que este código incluya:

+ Preprocesamiento de datos con StandardScaler
+ Selección de mejores categorías (investiga cuál es la mejor función de cálculo)
+ Pipelines
+ Evaluación de modelo

Suena fácil, pero te aseguro que poner todo junto en un mismo proyecto, puede hacer que las cosas se compliquen un poco.

Te deseo lo mejor como siempre, esperando que este proyecto implique un buen tiempo de diversión y resolución de problemas.

In [17]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Cargar el dataset de diabetes
diabetes = datasets.load_diabetes()

# Convertir a DataFrame para facilitar el análisis exploratorio
diabetes_df = pd.DataFrame(data=np.c_[diabetes['data'], diabetes['target']],
                           columns=diabetes['feature_names'] + ['target'])

# Convertir 'target' en categorías para clasificación
diabetes_df['target'] = (diabetes_df['target'] > diabetes_df['target'].median()).astype(int)

# División de datos en conjuntos de entrenamiento y prueba
X = diabetes_df.drop('target', axis=1)
y = diabetes_df['target']
X_entrena, X_prueba, y_entrena, y_prueba = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo RandomForest
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_entrena, y_entrena)

# Realizar predicciones con el conjunto de prueba
predicciones = modelo.predict(X_prueba)

# Evaluación del modelo
puntaje = modelo.score(X_prueba, y_prueba)
print(f"\nPrecisión del modelo: {puntaje:.2f}")



Precisión del modelo: 0.72


### Código optimizado

In [58]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import SelectKBest, f_classif

# Cargar el dataset de diabetes
diabetes = datasets.load_diabetes()

# Convertir a DataFrame para facilitar el análisis exploratorio
diabetes_df = pd.DataFrame(data=np.c_[diabetes['data'], diabetes['target']],
                           columns=diabetes['feature_names'] + ['target'])

# Convertir 'target' en categorías para clasificación
diabetes_df['target'] = (diabetes_df['target'] > diabetes_df['target'].median()).astype(int)

# Estandardizar los datos
escalador = StandardScaler()
diabetes_df = escalador.fit_transform(diabetes_df) 
# El escalador transforma los datos y devuelve un array numpy, por lo que es necesario convertirlo nuevamente a DataFrame
diabetes_df = pd.DataFrame(data=diabetes_df, columns=diabetes['feature_names'] + ['target'])

# División de datos en conjuntos de entrenamiento y prueba
X = diabetes_df.drop('target', axis=1)
y = diabetes_df['target']
X_entrena, X_prueba, y_entrena, y_prueba = train_test_split(X, y, test_size=0.2, random_state=42)

# Selección de mejores características
selector = SelectKBest(score_func=f_classif, k=4)
X_entrena_seleccionada = selector.fit_transform(X_entrena, y_entrena)
X_prueba_seleccionada = selector.transform(X_prueba)

# Crear y entrenar el modelo RandomForest
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_entrena_seleccionada, y_entrena)

# Realizar predicciones con el conjunto de prueba
predicciones = modelo.predict(X_prueba_seleccionada)

# Evaluación del modelo
puntaje = modelo.score(X_prueba_seleccionada, y_prueba)
print(f"\nPrecisión del modelo: {puntaje:.2f}")


Precisión del modelo: 0.74


In [ ]:
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import make_pipeline
    
# Cargar el dataset de diabetes
diabetes = datasets.load_diabetes()

# Convertir a DataFrame para facilitar el análisis exploratorio
diabetes_df = pd.DataFrame(data=np.c_[diabetes['data'], diabetes['target']],
                           columns=diabetes['feature_names'] + ['target'])

# Convertir 'target' en categorías para clasificación
diabetes_df['target'] = (diabetes_df['target'] > diabetes_df['target'].median()).astype(int)

# División de datos en conjuntos de entrenamiento y prueba
X = diabetes_df.drop('target', axis=1)
y = diabetes_df['target']
X_entrena, X_prueba, y_entrena, y_prueba = train_test_split(X, y, test_size=0.2, random_state=42)

selector = SelectKBest(score_func=f_classif, k=4)
X_entrena_seleccionada = selector.fit_transform(X_entrena, y_entrena)
X_prueba_seleccionada = selector.transform(X_prueba)

pipeline = make_pipeline(
    StandardScaler(),
    RandomForestClassifier(n_estimators=100, random_state=42)
)

pipeline.fit(X_entrena_seleccionada, y_entrena)

# Realizar predicciones con el conjunto de prueba y Evaluación del modelo
predicciones = pipeline.predict(X_prueba_seleccionada)
puntaje = pipeline.score(X_prueba_seleccionada, y_prueba)
print(f"\nPrecisión del modelo: {puntaje:.2f}") 

# Validación cruzada
puntaje = cross_val_score(pipeline, X, y, cv=5)
print(f"Exactitud de cada partición: {puntaje}")
print(f"Media de la exactitud: {puntaje.mean()}")


Precisión del modelo: 0.74
Exactitud de cada partición: [0.69662921 0.75280899 0.64772727 0.67045455 0.65909091]
Media de la exactitud: 0.6853421859039837
